In [43]:
# Import necessary libraries
import pandas as pd
import plotly.graph_objects as go
import dalmatian
from google.cloud import storage
import matplotlib.pyplot as plt
import networkx as nx
from networkx.drawing.nx_pydot import graphviz_layout

import numpy as np

from plotly.subplots import make_subplots

import sys
import os
from fludarabine import model_helper 
from fludarabine.helper import *
from fludarabine.plotly_helper import *

In [44]:
# Input patient information
patient_id = 'GCLL-0295'
wbc_file = '../../../data/gcll/CLL8 CLL cell count 05222025 UPDATED.csv'
treatment_file = '../../../data/gcll/GCLL_treatment.txt'
UMI_start = 1
UMI_end = 5
treatment_start = 0


In [45]:
# Manually verify the sample list order is correct

In [46]:
sample_list = ['GCLL-0295-T-01','RP-1895_PDO-31926_SM-NBY4X_GCLL-0295-T-03_v1_Custom_Selection_OnPrem', 'RP-1895_PDO-31926_SM-NBY4Z_GCLL-0295-T-06_v1_Custom_Selection_OnPrem', 'RP-1895_PDO-31926_SM-NBY5L_GCLL-0295-T-07_v1_Custom_Selection_OnPrem', 'RP-1895_PDO-31926_SM-NBY5C_GCLL-0295-T-08_v1_Custom_Selection_OnPrem', 'GCLL-0295-T-02']

In [47]:
# Output file

In [86]:
outputfile_name = "../Outputs/CLL_subplot_combined_0295_wbc_updated.html"

In [49]:


# Function to load data
def load_data(patient_id, wbc_file, treatment_file):
    # Load WBC and treatment data
    wbc_df = pd.read_csv(wbc_file)
    treatment_df = pd.read_csv(treatment_file, sep='\t')
    
    # Get input files from Terra through dalmatian
    workspace = 'broad-firecloud-ibmwatson/TAG_CLL_Clonal_Kinetic_UMI_PrAN'
    wm = dalmatian.WorkspaceManager(workspace)
    participants = wm.get_participants()
    
    # Load additional data for the patient
    cluster_CCF_df = pd.read_csv(participants.loc[patient_id]['cluster_ccfs'], sep='\t')
    abundance_df = pd.read_csv(participants.loc[patient_id]['abundances_tsv'], sep='\t')
    mcmc_df = pd.read_csv(participants.loc[patient_id]['cell_population_mcmc_trace'], sep='\t')
    tree_df = pd.read_csv(participants.loc[patient_id]['tree_tsv'], sep='\t')
    
    return wbc_df, treatment_df, cluster_CCF_df, abundance_df, mcmc_df, tree_df

# Function to filter data for a specific patient
def filter_patient_data(wbc_df, patient_id):
    wbc_df_patient = wbc_df[wbc_df['Patient'] == patient_id]
    wbc_df_patient.reset_index(drop=True, inplace=True)
    sample_list = wbc_df_patient['Timepoint'].dropna().tolist()
    times_sample = [int(i) for i in wbc_df_patient.loc[wbc_df_patient.Sample.notna(), 'Time_since_start_tx'].tolist()]
    wbc_all = [float(i) for i in wbc_df_patient['WBC'].tolist()]
    
    
    CLL_count = [float(i) for i in wbc_df_patient.loc[wbc_df_patient.Sample.notna(), 'CLL count estm'].tolist()]
    all_times = [int(i) for i in wbc_df_patient['Time_since_start_tx'].to_list()]
    
    return wbc_df_patient, times_sample, CLL_count, wbc_all,all_times





Update patient specific information in the below cell 

In [85]:
end_treatment_df = pd.read_csv("../../../data/gcll/CLL8_end_treatment.csv")

In [51]:
# Get the treatment duration 

treatment_end = end_treatment_df[end_treatment_df.Patient == patient_id].Treatment_duration.iloc[0]

In [52]:
treatment_end

149

In [53]:

wbc_df, treatment_df, cluster_CCF_df, abundance_df, mcmc_df, tree_df = load_data(patient_id, wbc_file, treatment_file)

# Filter data for the specific patient
wbc_df_patient, times_sample, CLL_count_sample,wbc_count, all_times = filter_patient_data(wbc_df, patient_id)

# Plot CLL count over time and save as HTML
CLL_plot_html = plot_CLL_count(patient_id, times_sample, CLL_count_sample, UMI_start, UMI_end, treatment_start, treatment_end, )
    




In [54]:
wbc_count

[176.33, 2.6, 3.2, 2.5, 3.33, 3.33, 4.7, 8.1, 11.66, 324.0]

In [55]:
wbc_table_html = plot_metadata_table(wbc_df_patient, patient_id )

In [56]:
 plot_tree_html = plot_tree_plotly(tree_df, 0)

In [57]:
 plot_ccf_html = plot_ccf(cluster_CCF_df, times_sample, treatment_df,)

In [58]:
branch_annotations = {
    1: "SF3B1_p.K700E ",
    2: "SAMHD1_p.R226L, del(13q)", 
    3: "DIS3_p.D487V",

}

In [59]:
ccf_tree_html = plot_ccf_tree_combined(
    tree_df=tree_df,
    tree_selected=0,
    ccf_df=cluster_CCF_df,
    times_sample=times_sample,
    treatment_df=treatment_df,
    branch_annotations=branch_annotations

)

SAMHD1_p.R226L<br>del(13q)


In [60]:
cluster_list, cluster_abundance = model_helper.get_abundance(abundance_df, mcmc_df, sample_list)
subclone_sample, log_subclone_sample = model_helper.calc_subclone(CLL_count_sample, cluster_abundance, cluster_list)
all_abundance = model_helper.get_all_abundance(cluster_list, mcmc_df, sample_list, times_sample, )

In [61]:
#  Add noise to all abundance

import random 
random.seed(42)
def add_uniform_noise_and_normalize(lst, low=0, high=0.01):
    
    # Add uniform noise
    noisy_lst = [x + random.uniform(low, high) for x in lst]

    
    # Normalize the list
    total_sum = sum(noisy_lst)
    
    normalized_lst = [x / total_sum for x in noisy_lst]

    return normalized_lst


subclone_cluster_iter = {}
# Iterate through each cluster
for cluster, iterations in all_abundance.items():
   
    # Iterate through each iteration in the cluster
    for iteration, value in iterations.items():
        # If the iteration is not in the new dict, add it
        if iteration not in subclone_cluster_iter:
            subclone_cluster_iter[iteration] = {}
        # Add the cluster and its value to this iteration
        subclone_cluster_iter[iteration][cluster] = value

        
        
noise_added_iter = {}

for iteration, abundances in subclone_cluster_iter.items():
    
    transposed_data = {i: list(values) for i, values in enumerate(zip(*abundances.values()), start=1)}
    
    noise_added_iter[iteration] = {}
    for time_point, abundance in transposed_data.items():
        
#         print(time_point, abundance)
        noise_abundance = add_uniform_noise_and_normalize(abundance)
    
#         print(noise_abundance)

        noise_added_iter[iteration][time_point] = add_uniform_noise_and_normalize(abundance)

    
noise_added_all_abundance = {}
for iteration, abundances in noise_added_iter.items():
    original_format = {i + 1: list(values) for i, values in enumerate(zip(*abundances.values()))}
    noise_added_all_abundance[iteration] = original_format
    
    
noise_added_original_all_abundances = {}

for iteration, abundances in noise_added_all_abundance.items():
    
    for cluster, abundance_value in abundances.items():
        if cluster not in noise_added_original_all_abundances:
            noise_added_original_all_abundances[cluster] = {}
        noise_added_original_all_abundances[cluster][iteration] = abundance_value
            

In [62]:
subclone_sample_mcmc_with_uniform_noise, log_subclone_sample_mcmc_with_uniform_noise = model_helper.calc_subclone(CLL_count_sample, noise_added_original_all_abundances, cluster_list, input_type = "mcmc")

In [63]:
times_aft_tx = [x for x in all_times if x > treatment_end]
times_aft_tx.insert(0, treatment_end)

# Set the extrapolate time after treatment
extrapolate_start_idx = 1

In [64]:
times_aft_tx

[149, 182, 246, 357, 453, 537, 628, 733, 1115]

In [65]:
# Generate Plotly plot
subclone_plot_html = plot_subclones(cluster_list, times_sample, CLL_count_sample, log_subclone_sample, extrapolate_start_idx, times_aft_tx, treatment_df, treatment_end)


In [66]:
linear_model_mcmc_html = plot_linear_model_mcmc(cluster_list,times_sample,  CLL_count_sample, log_subclone_sample_mcmc_with_uniform_noise,extrapolate_start_idx,times_aft_tx,treatment_df, treatment_end)  

In [67]:
# Convert the column "CLL count estm" to float 
wbc_df_patient['CLL count estm'] = wbc_df_patient['CLL count estm'].astype(float) 
times_sliced_aft = [int(i) for i in wbc_df_patient[wbc_df_patient['CLL count estm'] > 0]['Time_since_start_tx'].values  if int(i) > 0]

/var/folders/x6/7zg8fddx4dlgx2lv4sv34_g40000gp/T/ipykernel_11848/981427378.py:2: SettingWithCopyWarning:


A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy



In [68]:
times_aft_tx

[149, 182, 246, 357, 453, 537, 628, 733, 1115]

In [69]:
# Needs to manually check times_sliced_aft

In [71]:
times_sliced_aft = times_sliced_aft[1:]

In [72]:
times_sliced_aft

[182, 246, 357, 453, 537, 628, 733, 1115]

In [80]:
len(times_sliced_aft)

8

In [73]:
index_samples_model = slice(1,6)

# index_samples_model is the index of the samples after treatment. For instance, slice(1,7) means sample 2, 3, 4, 5,6, 7

X, y = create_inputs(times_sliced_aft, log_subclone_sample_mcmc_with_uniform_noise,2, index_samples_model, times_sample)



In [75]:
y

[None,
 None,
 None,
 None,
 array([-3.58342081, -5.77689919, -4.30875239, -6.74167712]),
 array([-2.35706877, -1.06398   , -4.93428498, -5.26516943]),
 array([-1.31748074,  0.16522793, -4.17936397, -3.54620385]),
 array([ 0.47763404,  1.5801554 , -2.64519208, -2.33381304])]

In [76]:
wbc_aft_tx = wbc_df_patient[wbc_df_patient.Time_since_start_tx > treatment_end]['CLL count estm'].to_list()
wbc_model = [i for i in wbc_aft_tx if i > 0]

In [77]:
wbc_model

[0.00133297,
 0.00277928,
 0.02679978,
 0.09401632,
 0.45196039,
 1.49101682,
 6.63322958,
 291.6]

In [78]:
len(wbc_model)

8

In [79]:
n_clusters = len(cluster_list)
logsumexp_points = np.log(wbc_model)
model = MultiClusterLinearRegression(n_clusters, X, y)
model.fit(logsumexp_points)

In [81]:
# Generate Plotly plot
subclone_plot_new_model_html = plot_subclones_new_model(cluster_list, times_sample, wbc_model, log_subclone_sample, extrapolate_start_idx, times_aft_tx, times_sliced_aft, treatment_df,treatment_end, model )


In [82]:
plot_mcmc_new_model_html = plot_mcmc_model(cluster_list, index_samples_model, times_aft_tx, times_sample, times_sliced_aft, sample_list, wbc_model, log_subclone_sample_mcmc_with_uniform_noise, treatment_df, treatment_end)

In [87]:
create_html_file([wbc_table_html, plot_ccf_html, ccf_tree_html, CLL_plot_html, subclone_plot_html,linear_model_mcmc_html,subclone_plot_new_model_html, plot_mcmc_new_model_html], output_file=outputfile_name)

HTML file saved as ../Outputs/CLL_subplot_combined_0295_wbc_updated.html
